# 0.0 Imports

In [4]:
import pandas             as pd
import numpy              as np
#import xgboost            as xgb
import scikitplot         as skplt

import matplotlib.pyplot  as plt
import seaborn            as sns

from IPython.display         import Image
from scipy                   import stats            as ss
from sklearn                 import preprocessing    as pp
from sklearn                 import model_selection  as ms
from sklearn                 import ensemble         as en
from sklearn                 import metrics          as m
from scikitplot              import metrics          as mt
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.ensemble     import RandomForestClassifier
#from boruta               import BorutaPy
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors    import KNeighborsClassifier
from sklearn              import svm
#from lightgbm             import LGBMClassifier
from sklearn.naive_bayes  import GaussianNB
#from skopt                import forest_minimize
#from skopt.plots          import plot_convergence
from flask                import Flask, request, Response

import warnings
import pickle
warnings.filterwarnings("ignore")

%matplotlib inline

## 0.1 Helper Functions

## 0.2 Loading Data

In [7]:
#data_kaggle = pd.read_csv('C:/Users/Guilherme/Repos/Dataset pa002/dataset_kaggle_block_fraud_company.csv')

In [11]:
data_kaggle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


- Memory usage is too high (500MB+), there are many data to manipulate in a local machine. A split strategy will be applied to split data for less memory use

### 0.2.1 Data split 

In [9]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_1_index, train_2_index in split.split(data_kaggle, data_kaggle['isFraud']):
    strat_train_1_set = data_kaggle.loc[train_1_index]
    strat_train_2_set = data_kaggle.loc[train_2_index]

In [10]:
strat_train_2_set.to_csv('C:/Users/Guilherme/Repos/Dataset pa002/dataset_sampled.csv', index=False)
strat_train_2_set.info()

- Now memory is 116 MB of usage, more acceptable to use in a local machine to manipulate data and process it.

### 0.2.1 Loading splited data

In [14]:
data_splited = pd.read_csv('C:/Users/Guilherme/Repos/Dataset pa002/dataset_sampled.csv')

#### 0.2.1.1 Slipt data in dataset (train and validation) and test

In [15]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for test_1_index, test_2_index in split.split(data_splited, data_splited['isFraud']):
    dataset = data_splited.loc[test_1_index]
    dataset_test = data_splited.loc[test_2_index]

In [20]:
dataset['isFraud'].value_counts()#normalize=True)

0    1016705
1       1314
Name: isFraud, dtype: int64

In [21]:
dataset_test['isFraud'].value_counts()#normalize=True)

0    254176
1       329
Name: isFraud, dtype: int64

In [ ]:
a